In [ ]:
# Notebook: 03_sql_analysis
# Purpose: Perform SQL analysis on Gold layer

import os

# Configuration
storage_account = "dota2lakehouse"
container = "data"
storage_account_key = os.environ.get("AZURE_STORAGE_KEY")

spark.conf.set(
    f"fs.azure.account.key.{storage_account}.dfs.core.windows.net",
    storage_account_key
)

GOLD_PATH = f"abfss://{container}@{storage_account}.dfs.core.windows.net/gold"


-- QUERY 1: Top 10 Players by KDA (Minimum 10 matches)
SELECT 
    account_id,
    last_known_name,
    total_matches,
    avg_kda,
    avg_kills,
    avg_deaths,
    avg_assists,
    win_rate,
    kda_rank
FROM esports_gold.player_stats
WHERE total_matches >= 10
ORDER BY avg_kda DESC
LIMIT 10

-- QUERY 2: Top 10 Players by Win Rate (Minimum 20 matches)
SELECT 
    account_id,
    last_known_name,
    total_matches,
    total_wins,
    win_rate,
    avg_kda,
    avg_gpm
FROM esports_gold.player_stats
WHERE total_matches >= 20
ORDER BY win_rate DESC
LIMIT 10

-- QUERY 3: Hero Meta Analysis - Most Picked/Banned Heroes
SELECT 
    hero_id,
    times_played,
    win_rate,
    pick_count,
    ban_count,
    total_presence,
    presence_rate,
    meta_tier,
    avg_kda,
    avg_gpm
FROM esports_gold.hero_stats
ORDER BY presence_rate DESC
LIMIT 15

-- QUERY 4: Match Duration Correlation with Win Rate
SELECT 
    CASE 
        WHEN duration_minutes < 25 THEN '1. Very Short (<25min)'
        WHEN duration_minutes < 35 THEN '2. Short (25-35min)'
        WHEN duration_minutes < 45 THEN '3. Medium (35-45min)'
        WHEN duration_minutes < 55 THEN '4. Long (45-55min)'
        ELSE '5. Very Long (>55min)'
    END as duration_category,
    COUNT(*) as match_count,
    ROUND(AVG(CASE WHEN win = 1 THEN 1.0 ELSE 0.0 END) * 100, 2) as win_rate,
    ROUND(AVG(kills), 2) as avg_kills,
    ROUND(AVG(gold_per_min), 2) as avg_gpm
FROM esports_gold.ml_features
GROUP BY 1
ORDER BY 1

-- QUERY 5: Performance Trends Over Time (Monthly)
SELECT 
    DATE_TRUNC('month', match_date) as month,
    COUNT(*) as match_count,
    ROUND(AVG(avg_duration), 2) as avg_duration,
    ROUND(AVG(avg_total_kills), 2) as avg_kills_per_match,
    ROUND(AVG(radiant_win_rate), 2) as avg_radiant_win_rate
FROM esports_gold.daily_stats
GROUP BY 1
ORDER BY 1

-- QUERY 6: Hero Performance by Lane/Role
SELECT 
    lane,
    lane_role,
    COUNT(DISTINCT hero_id) as unique_heroes,
    COUNT(*) as total_games,
    ROUND(AVG(kda_calculated), 2) as avg_kda,
    ROUND(AVG(gold_per_min), 2) as avg_gpm,
    ROUND(AVG(CASE WHEN win = 1 THEN 1.0 ELSE 0.0 END) * 100, 2) as win_rate
FROM esports_gold.ml_features
WHERE lane IS NOT NULL AND lane_role IS NOT NULL
GROUP BY lane, lane_role
ORDER BY total_games DESC
```
